In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [117]:
pods = pd.read_csv('podcasts.csv', index_col='name')

In [118]:
pods

,link
name,
Kim Komando Daily Tech Update,https://feeds.libsyn.com/115469/rss
WSJ Tech News Briefing,https://video-api.wsj.com/podcast/rss/wsj/tec...
Gadget Lab,https://publicfeeds.net/f/5901/gadget-lab
TechONTAPPodcast,https://feeds.soundcloud.com/users/soundcloud...
Linux Out Loud,https://feeds.fireside.fm/dlnxtend/rss
Late Night Linux Family All Episodes,https://latenightlinux.com/feed/all
Destination Linux,https://destinationlinux.org/feed/mp3/
LINUX Unplugged,https://feeds.fireside.fm/linuxunplugged/rss
Enterprise Linux Security,https://enterpriselinuxsecurity.show/feed/pod...


In [119]:
for i, url in pods.itertuples():
  print(url)

 https://feeds.libsyn.com/115469/rss
 https://video-api.wsj.com/podcast/rss/wsj/tech-news-briefing
 https://publicfeeds.net/f/5901/gadget-lab
 https://feeds.soundcloud.com/users/soundcloud:users:164421460/sounds.rss
 https://feeds.fireside.fm/dlnxtend/rss
 https://latenightlinux.com/feed/all
 https://destinationlinux.org/feed/mp3/
 https://feeds.fireside.fm/linuxunplugged/rss
 https://enterpriselinuxsecurity.show/feed/podcast/
 https://latenightlinux.com/feed/extra
 https://lhspodcast.info/category/podcast-mp3/feed/
 https://tuxdigital.com/feed/thisweekinlinux-mp3
 https://linuxlads.com/feed_mp3.rss
 https://feeds.twit.tv/uls.xml
 https://feed.syntax.fm/rss
 https://changelog.com/jsparty/feed
 https://feeds.feedwrench.com/js-jabber.rss
 https://softwareengineeringdaily.com/category/javascript/feed/
 https://anchor.fm/s/e329dea0/podcast/rss
 https://realpython.com/podcasts/rpp/feed
 http://adafruit-podcasts.s3.amazonaws.com/python_on_hardware/podcast.xml
 https://pythonbytes.fm/episodes

In [6]:
# pods.to_csv('pods.csv',index=True, index_label='name')

In [120]:
rssFeedEntries = []

for i, url in pods.itertuples():
  req = requests.get( url, allow_redirects=False )
  reqCode = req.status_code
  soup = BeautifulSoup(req.content, 'xml')
  entries = soup.findAll('item')
  rssFeedEntries.append([i,req, reqCode, entries ])
  req.close()
  soup.clear()
  

In [121]:
for i, r, rc, feed in rssFeedEntries:
  print( f'{i:75} {r} {rc:5} {len(feed):8}' )

Kim Komando Daily Tech Update                                               <Response [200]>   200     2854
WSJ Tech News Briefing                                                      <Response [200]>   200       20
Gadget Lab                                                                  <Response [200]>   200      255
TechONTAPPodcast                                                            <Response [200]>   200      405
Linux Out Loud                                                              <Response [200]>   200      180
Late Night Linux Family All Episodes                                        <Response [200]>   200      439
Destination Linux                                                           <Response [200]>   200      300
LINUX Unplugged                                                             <Response [200]>   200      547
Enterprise Linux Security                                                   <Response [200]>   200       78
Linux Dev Time              

In [122]:
rp = pd.DataFrame(rssFeedEntries, columns=['name','response', 'response code','entries'])

In [123]:
type(rp['entries'][0])

bs4.element.ResultSet

In [124]:
type(rp['response'][0])

requests.models.Response

In [125]:
requests.codes['OK']

200

In [126]:
rp.response[0].reason

'OK'

In [127]:
for r in rp.response:
  print (r.reason)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
Moved Permanently
Forbidden
OK
OK
OK
OK
OK
Found
Found
OK
Not Found
OK


In [15]:
successfulEntries = rp[rp['response code'] == 200]['entries']

In [16]:
# print(successfulEntries[0])

for i in successfulEntries[0]:
  print( 'title: ', i.title.text )
  print( 'published date: ',  i.pubDate.text )
  print( 'description: ',  i.description.text )
  print( 'image: ', i.image['href'] if i.image else 'no image found')
  print( 'link: ', i.enclosure['url'] if i.enclosure else 'no link found', '\n' )

title:  2 signs you're getting AirTag stalked
published date:  Sat, 03 Feb 2024 19:00:00 +0000
description:  <p>Feeling watched? Find out how to catch sneaky AirTag trackers with these tips. </p>
image:  https://static.libsyn.com/p/assets/4/f/5/5/4f55c2676477d2ebe5bbc093207a2619/0001-3703502264337991165.jpg
link:  https://pscrb.fm/rss/p/traffic.libsyn.com/secure/consumertechupdate/2024_0203_DLH_POD.mp3?dest-id=656199 

title:  More bad news for EVs
published date:  Sat, 03 Feb 2024 10:00:00 +0000
description:  <p>The news just kept getting worse for the electric vehicle industry. Here's why customers aren't buying. </p>
image:  no image found
link:  https://pscrb.fm/rss/p/traffic.libsyn.com/secure/consumertechupdate/2024_0203_POD.mp3?dest-id=656199 

title:  Taylor Swift's deepfake fiasco
published date:  Fri, 02 Feb 2024 19:00:00 +0000
description:  <p>Found a deepfake of yourself online? Here's your game plan if it happens to you.</p>
image:  https://static.libsyn.com/p/assets/7/3/8/

In [17]:
sample = successfulEntries[0]

In [20]:
podcast = requests.get(sample[0].enclosure['url'])

In [25]:
# try:
#   with open(sample[1].title.text + '.mp3', 'xb' ) as f:
#     f.write(podcast.content)
# except(FileExistsError):
#   print( 'file already exists' )

In [87]:
# for i in successfulEntries[0]:
#   if i.enclosure:
#     try:
#       with open( './podcasts/'+ ''.join( [c for c in i.title.text if c.isalnum() or c == ' ' ] ) + '.mp3', 'xb' ) as f:
#         f.write( requests.get(i.enclosure['url']).content )
#     except( FileExistsError ):
#       print( 'file already exists' )

file already exists
file already exists
file already exists
file already exists
file already exists
file already exists


KeyboardInterrupt: 

In [85]:
# s = 'Is your spouse spying on you?'
s= 'AI is 2024\'s "show me the mon?ey"'
print( s.replace("?",'') )

AI is 2024's "show me the money"


In [156]:
def downloadPod( url, name, dirname ):
  try:
    with open( './'+ dirname + '/' + ''.join( [c for c in name if c.isalnum() or c == ' ' ] ) + '.mp3', 'xb' ) as f:
      f.write( requests.get(url).content )
  except( FileExistsError ):
    print( 'file already exists' )
    return False
  return True

In [155]:
syntax = rp[ rp['name'] == 'Syntax' ]['entries']

for i in syntax:
  for j in range(10):
    print(i[j].enclosure['url'])
    downloadPod( i[j].enclosure['url'], i[j].title.text, 'syntax' )

https://traffic.libsyn.com/secure/syntax/Safari_is_the_new_Chrome_725.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Potluck_Is_Virtual_Dom_Bad.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/723_-_Svelte_5_Speed_Simplicity__Size.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_722.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_721.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_720.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_719fx.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_718.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_717.mp3?dest-id=532671
https://traffic.libsyn.com/secure/syntax/Syntax_-_716.mp3?dest-id=532671
